In [1]:
import pandas as pd
import sqlite3

In [2]:
orders = pd.read_csv("orders.csv")

In [3]:
users = pd.read_json("users.json")

In [4]:
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

with open("restaurants.sql", "r") as f:
    sql_script = f.read()

cursor.executescript(sql_script)

restaurants = pd.read_sql("SELECT * FROM restaurants", conn)

In [5]:
orders_users = pd.merge(
    orders,
    users,
    on="user_id",
    how="left"
)


In [6]:
final_df = pd.merge(
    orders_users,
    restaurants,
    on="restaurant_id",
    how="left"
)


In [7]:
final_df.to_csv("final_food_delivery_dataset.csv", index=False)


In [8]:

df = pd.read_csv("final_food_delivery_dataset.csv")

In [9]:
df.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [10]:
gold = df[df["membership"] == "Gold"]

In [13]:
gold.head() #gold membership top members

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
8,9,364,7,05-12-2023,953.30,Ruchi Biryani Punjabi,User_364,Chennai,Gold,Restaurant_7,Italian,3.2
11,12,884,423,27-10-2023,1484.65,Royal Tiffins Multicuisine,User_884,Pune,Gold,Restaurant_423,Indian,3.2
13,14,364,112,24-09-2023,898.24,Grand Cafe Punjabi,User_364,Chennai,Gold,Restaurant_112,Chinese,3.2


In [15]:
gold.groupby("city")["total_amount"].sum().sort_values(ascending=False) #top revenue

city
Chennai      1080909.79
Pune         1003012.32
Bangalore     994702.59
Hyderabad     896740.19
Name: total_amount, dtype: float64

In [16]:
df.groupby("cuisine")["total_amount"].mean().sort_values(ascending=False)

cuisine
Mexican    808.021344
Italian    799.448578
Indian     798.466011
Chinese    798.389020
Name: total_amount, dtype: float64

In [17]:
user_total = df.groupby("user_id")["total_amount"].sum()
high_value_users = user_total[user_total > 1000]
high_value_users.count()

np.int64(2544)

In [18]:
df.columns

Index(['order_id', 'user_id', 'restaurant_id', 'order_date', 'total_amount',
       'restaurant_name_x', 'name', 'city', 'membership', 'restaurant_name_y',
       'cuisine', 'rating'],
      dtype='object')

In [19]:
bins = [3.0, 3.5, 4.0, 4.5, 5.0]
labels = ["3.0 – 3.5", "3.6 – 4.0", "4.1 – 4.5", "4.6 – 5.0"]

df["rating_range"] = pd.cut(
    df["rating"],
    bins=bins,
    labels=labels,
    include_lowest=True
)

In [21]:
df.groupby("rating_range")["total_amount"].sum().sort_values(ascending=False)  #highest ratings

<ipython-input-21-1e3cfeef280f>:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby("rating_range")["total_amount"].sum().sort_values(ascending=False)  #highest ratings


rating_range
4.6 – 5.0    2197030.75
3.0 – 3.5    2136772.70
4.1 – 4.5    1960326.26
3.6 – 4.0    1717494.41
Name: total_amount, dtype: float64

In [24]:
gold = df[df["membership"] == "Gold"]
gold.columns
gold.groupby("city")["total_amount"].mean().sort_values(ascending=False)  #city with highest avg order value among gold members

city
Chennai      808.459080
Hyderabad    806.421034
Bangalore    793.223756
Pune         781.162243
Name: total_amount, dtype: float64

In [26]:
restaurant_count = df.groupby("cuisine")["restaurant_id"].nunique()
revenue_by_cuisine = df.groupby("cuisine")["total_amount"].sum()
summary = pd.DataFrame({
    "restaurant_count": restaurant_count,
    "total_revenue": revenue_by_cuisine
}).sort_values("restaurant_count")      #lowest number of distinct restaurants

summary


,restaurant_count,total_revenue
cuisine,,
Chinese,120,1930504.65
Indian,126,1971412.58
Italian,126,2024203.80
Mexican,128,2085503.09


In [28]:

#percentage of total order placed by gold members
total_orders = len(df)
gold_orders = len(df[df["membership"] == "Gold"])
percentage = round((gold_orders / total_orders) * 100)
percentage


50

In [33]:
restaurant_stats = df.groupby("restaurant_name_x").agg(
    avg_order_value=("total_amount", "mean"),
    total_orders=("order_id", "count")
)
filtered = restaurant_stats[restaurant_stats["total_orders"] < 20]
filtered.sort_values("avg_order_value", ascending=False)
#df[["restaurant_name_x", "restaurant_name_y"]]

,avg_order_value,total_orders
restaurant_name_x,,
Hotel Dhaba Multicuisine,1040.222308,13
Sri Mess Punjabi,1029.180833,12
Ruchi Biryani Punjabi,1002.140625,16
Sri Delights Pure Veg,989.467222,18
Classic Kitchen Family Restaurant,973.167895,19
...,...,...
Annapurna Tiffins Punjabi,621.828947,19
Darbar Tiffins Non-Veg,596.815556,18
Darbar Restaurant Punjabi,589.972857,14


In [34]:

combo_revenue = df.groupby(["membership", "cuisine"])["total_amount"].sum().sort_values(ascending=False)
combo_revenue

membership  cuisine
Regular     Mexican    1072943.30
            Italian    1018424.75
Gold        Mexican    1012559.79
            Italian    1005779.05
Regular     Indian      992100.27
Gold        Indian      979312.31
            Chinese     977713.74
Regular     Chinese     952790.91
Name: total_amount, dtype: float64

In [35]:
df['order_date'] = pd.to_datetime(df['order_date'])
df['quarter'] = df['order_date'].dt.quarter
df['quarter'] = df['order_date'].dt.quarter
quarter_revenue = df.groupby('quarter')['total_amount'].sum().sort_values(ascending=False)
quarter_revenue

<ipython-input-35-778a2a328a8f>:1: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['order_date'] = pd.to_datetime(df['order_date'])


quarter
3    2037385.10
4    2018263.66
1    2010626.64
2    1945348.72
Name: total_amount, dtype: float64

In [38]:
gold_orders = df[df["membership"] == "Gold"]
total_gold_orders = len(gold_orders)
total_gold_orders   #orders of gold memmmbers


4987

In [37]:
df.columns
hyderabad_orders = df[df["city"] == "Hyderabad"]
total_revenue = hyderabad_orders["total_amount"].sum()
total_revenue_rounded = round(total_revenue)
total_revenue_rounded

1889367

In [39]:
distinct_users = df["user_id"].nunique()
distinct_users

2883

In [40]:
gold_orders = df[df["membership"] == "Gold"]
avg_order_value_gold = round(gold_orders["total_amount"].mean(), 2)
avg_order_value_gold


np.float64(797.15)

In [41]:
high_rating_orders = df[df["rating"] >= 4.5]
num_high_rating_orders = len(high_rating_orders)
num_high_rating_orders

3374

In [42]:
gold_orders = df[df["membership"] == "Gold"]
top_city = gold_orders.groupby("city")["total_amount"].sum().idxmax()
top_city
orders_top_city = len(gold_orders[gold_orders["city"] == top_city])
orders_top_city

1337